In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/depth_scale.npy
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/background_depth.png
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/depth.mp4
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/inverse_intrinsic.npy
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/output.json
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/rot.npy
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/times.txt
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/color.mp4
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/background.png
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000016/mask.png
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000028/depth_scale.npy
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000028/background_depth.png
/kaggle/input/pigbehaviordetection/annotated/2019_11_11/000028/depth.mp4
/

In [2]:
!apt-get update
!apt-get install -y build-essential libopencv-dev git

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,722 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,665 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,546 kB]
Get:13 https://ppa.launch

In [3]:
import os

YOLO_ROOT = "/kaggle/working/yolo"
for subdir in ["images/train","images/val","labels/train","labels/val","backup"]:
    os.makedirs(os.path.join(YOLO_ROOT, subdir), exist_ok=True)

In [4]:
import json, cv2
from pathlib import Path

ANNOT_DIR = Path("/kaggle/input/pigbehaviordetection/annotated")
YOLO      = Path(YOLO_ROOT)
train_seqs = ['000002','000009','000016','000028','000036','000033','000010']
val_seqs   = ['000113','000005','000208','000060','000078']

In [5]:
def to_yolo(bbox, W, H):
    x,y,w,h = bbox["x"], bbox["y"], bbox["width"], bbox["height"]
    cx, cy = x + w/2, y + h/2
    return f"0 {cx/W:.6f} {cy/H:.6f} {w/W:.6f} {h/H:.6f}\n"

In [ ]:
# Open train.txt / val.txt for Darknet
train_list = open(YOLO/"train.txt","w")
val_list   = open(YOLO/"val.txt","w")

for split, seqs, list_f in [("train",train_seqs,train_list),("val",val_seqs,val_list)]:
    for date_dir in ANNOT_DIR.iterdir():
        if not date_dir.is_dir(): continue
        for seq in seqs:
            clip = date_dir/seq
            if not clip.exists(): continue

            # Load ground-truth JSON
            ann = json.load(open(clip/"output.json"))

    
            cap = cv2.VideoCapture(str(clip/"color.mp4"))
            W = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
            H = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

            # Build map raw_frame_idx -> list of bboxes
            frame_map = {}
            for obj in ann["objects"]:
                for fr in obj["frames"]:
                    raw = fr["frameNumber"] * 3
                    frame_map.setdefault(raw, []).append(fr["bbox"])

            # For each raw frame with annotations...
            for raw_idx, bboxes in frame_map.items():
                img_path = YOLO/f"images/{split}/{seq}_{raw_idx:04d}.jpg"
                lbl_path = YOLO/f"labels/{split}/{seq}_{raw_idx:04d}.txt"

                # Save image if not already
                if not img_path.exists():
                    cap.set(cv2.CAP_PROP_POS_FRAMES, raw_idx)
                    ret, frame = cap.read()
                    cv2.imwrite(str(img_path), frame)

                # Write label file
                with open(lbl_path, "w") as lf:
                    for bb in bboxes:
                        lf.write(to_yolo(bb, W, H))

                # Append image absolute path to train.txt / val.txt
                list_f.write(str(img_path.resolve()) + "\n")

            cap.release()

train_list.close()
val_list.close()
print("✅ YOLO data conversion complete")

✅ YOLO data conversion complete


In [ ]:
# with open(YOLO/"obj.names","w") as f:
#     f.write("pig\n")

# with open(YOLO/"obj.data","w") as f:
#     f.write("classes = 1\n")
#     f.write(f"train  = {YOLO}/train.txt\n")
#     f.write(f"valid  = {YOLO}/val.txt\n")
#     f.write("names  = obj.names\n")
#     f.write("backup = backup/\n")


# CFG_BASE = Path("/kaggle/working/cfg/yolov3.cfg")
# CFG_NEW  = YOLO/"yolov3_pig.cfg"
# !cp "{CFG_BASE}" "{CFG_NEW}"
# # change classes=80 → classes=1, filters=255 → filters=18
# !sed -i 's/classes=80/classes=1/' "{CFG_NEW}"
# !sed -i 's/filters=255/filters=18/'  "{CFG_NEW}"

# print("✅ Created YOLO config:", CFG_NEW)

In [8]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-

In [9]:
from pathlib import Path
import yaml

In [10]:
YOLO_DIR = Path("/kaggle/working/yolo")
data_yaml = YOLO_DIR/"data.yaml"

In [11]:
config = {
    'path': str(YOLO_DIR),
    'train': 'images/train',
    'val':   'images/val',
    'nc':    1,
    'names': ['pig']
}
with open(data_yaml, 'w') as f:
    yaml.dump(config, f)
print("Wrote data config to:", data_yaml)

Wrote data config to: /kaggle/working/yolo/data.yaml


In [12]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [13]:
model = YOLO('yolov8n.pt')  

100%|██████████| 6.25M/6.25M [00:00<00:00, 79.9MB/s]


In [ ]:
results = model.train(
    data=str(data_yaml),
    epochs=50,
    imgsz=416,
    batch=16,            
    save=True,           
    project=str(YOLO_DIR/"runs"),
    name="yolov8_pig"
)

Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_pig, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 72.5MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2329.5±1556.8 MB/s, size: 318.8 KB)


...


50 epochs completed in 0.427 hours.
Optimizer stripped from /kaggle/working/yolo/runs/yolov8_pig/weights/last.pt, 6.2MB
Optimizer stripped from /kaggle/working/yolo/runs/yolov8_pig/weights/best.pt, 6.2MB

Validating /kaggle/working/yolo/runs/yolov8_pig/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:09<00:00,  5.88it/s]


                   all       1744       4091      0.354      0.817      0.431      0.221


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /kaggle/working/yolo/runs/yolov8_pig


In [15]:
metrics = model.val()  
print(metrics)  

Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3491.7±1023.6 MB/s, size: 361.6 KB)


val: Scanning /kaggle/working/yolo/labels/val.cache... 1744 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1744/1744 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:09<00:00, 11.55it/s]


                   all       1744       4091      0.353      0.816      0.431      0.221


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /kaggle/working/yolo/runs/yolov8_pig2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a437ae68bd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038, 

In [16]:
preds = model.predict(
    source=str(YOLO_DIR/"images/val")+"/**/*.jpg",
    save=True,
    imgsz=416,
    conf=0.25,   # confidence threshold
    project=str(YOLO_DIR/"runs"),
    name="predict",
    exist_ok=True
)
print("Saved predictions to:", YOLO_DIR/"runs"/"predict")


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1744 /kaggle/working/yolo/images/val/000005_0000.jpg: 256x416 5 pigs, 37.7ms
image 2/1744 /kaggle/working/yolo/images/val/000005_0021.jpg: 256x416 1 pig, 6.1ms
image 3/1744 /kaggle/working/yolo/images/val/000005_0033.jpg: 256x416 3 pigs, 5.9ms
image 4/1744 /kaggle/working/yolo/images/val/000005_0036.jpg: 256x416 2 pigs, 6.2ms
image 5/1744 /kaggle/working/yolo/images/val/000005_0039.jpg: 256x416 2 pigs, 5.9ms
image 6/1744 /kaggle/working/yolo/im

In [17]:
model.export(format='onnx',  
             imgsz=416,
             dynamic=True,
             batch=1,
             opset=12
)
print("Exported ONNX model to:", Path("models")/"best.onnx")

Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/kaggle/working/yolo/runs/yolov8_pig/weights/best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 5, 3549) (5.9 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.46', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.6/145.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 251.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 260.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 166.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 11.2s, installed 2 packages: ['onnxslim>=0.1.46', 'onnxruntime-gpu']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take ef

### Improved version

In [18]:
# !pip install -U ultralytics

In [19]:
# from ultralytics import YOLO
# from pathlib import Path
# import yaml

In [20]:

# YOLO_DIR   = Path("/kaggle/working/yolo")
# DATA_YAML  = YOLO_DIR/"data.yaml"
# RUNS_DIR   = YOLO_DIR/"runs"
# RUN_NAME   = "yolov8m_pig_improved"
# WEIGHTS    = RUNS_DIR/RUN_NAME/"weights"/"best.pt"

# # 3) Re-write data.yaml if needed
# config = {
#     'path': str(YOLO_DIR),
#     'train': 'images/train',
#     'val':   'images/val',
#     'nc':    1,
#     'names': ['pig']
# }
# with open(DATA_YAML,'w') as f:
#     yaml.dump(config, f)

# # 4) Load a medium-sized YOLOv8 backbone
# model = YOLO('yolov8m.pt')  # upgrade from nano to medium

In [21]:
# # 5) Train with stronger settings
# results = model.train(
#     data=str(DATA_YAML),
#     model='yolov8m.pt',
#     epochs=100,
#     imgsz=640,
#     batch=8,
#     optimizer='SGD',
#     lr0=0.01,
#     lrf=0.2,            # final LR = lr0 * lrf
#     momentum=0.937,
#     weight_decay=0.0005,
#     patience=50,        # early stop if no improvement
#     augment=True,         # stronger aug & hyperparams
#     project=str(RUNS_DIR),
#     name=RUN_NAME,
#     save=True,
#     device='0'
# )


In [22]:
# from ultralytics import YOLO
# val_model = YOLO(str(WEIGHTS))
# print("New val metrics:", val_model.val())

In [23]:

# # 6) Summarize your new metrics
# metrics = model.val()  
# print(metrics)



In [24]:
# # 7) Plot PR and loss curves
# # (Ultralytics automatically saves them under runs/<name>/)
# from IPython.display import display, Image
# display(Image(str(RUNS_DIR/RUN_NAME/'results.png')))